In [3]:
import pandas as pd
from bs4 import BeautifulSoup
import urllib
import pymysql
import calendar
import time
import json
from urllib.request import urlopen
from datetime import datetime
from threading import Timer


In [4]:
def read_naver(self, code, company, pages_to_fetch):
    """네이버에서 주식 시세를 읽어서 데이터프레임으로 반환"""
    try:
        url = f"http://finance.naver.com/item/sise_day.nhn?code={code}"
        with urlopen(url) as doc:
            if doc is None:
                return None
            html = BeautifulSoup(doc, "lxml")
            pgrr = html.find("td", class_="pgRR")
            if pgrr is None:
                return None
            s = str(pgrr.a["href"]).split('=')
            lastpage = s[-1]  # 일별 시세의 마지막
        df = pd.DataFrame()
        pages = min(int(lastpage), pages_to_fetch)
        for page in range(1, pages + 1):
            pg_url = '{}&page={}'.format(url, page)
            df = df.append(pd.read_html(pg_url, header=0)[0])
            tmnow = datetime.now().strftime('%Y-%m-%d %H:%M')
            print('[{}] {} ({}) : {:04d}/{:04d} pages are downloading...'.
                  format(tmnow, company, code, page, pages), end="\r")
        df = df.rename(columns={'날짜': 'date', '종가': 'close', '전일비': 'diff',
                       '시가': 'open', '고가': 'high', '저가': 'low', '거래량': 'volume'})
        df['date'] = df['date'].replace('.', '-')
        df = df.dropna()
        df[['close', 'diff', 'open', 'high', 'low', 'volume']] = df[['close',
                                                                     'diff', 'open', 'high', 'low', 'volume']].astype(int)
        df = df[['date', 'open', 'high', 'low', 'close', 'diff', 'volume']]
    except Exception as e:
        print('Exception occured :', str(e))
        return None
    return df


In [5]:
d = {'col1': [1, 2], 'col2': [3, 4]}
df = pd.DataFrame(data=d)
df.dtypes

col1    int64
col2    int64
dtype: object

In [7]:
df.dtypes

col1    int64
col2    int64
dtype: object

In [12]:
df.astype('float').dtypes

col1    float64
col2    float64
dtype: object

In [13]:
def replace_into_db(self, df, num, code, company):
        """네이버에서 읽어온 주식 시세를 DB에 REPLACE"""
        with self.conn.cursor() as curs:
            for r in df.itertuples():
                sql = f"REPLACE INTO daily_price VALUES ('{code}', "\
                    f"'{r.date}', {r.open}, {r.high}, {r.low}, {r.close}, "\
                    f"{r.diff}, {r.volume})"
                curs.execute(sql)
            self.conn.commit()
            print('[{}] #{:04d} {} ({}) : {} rows > REPLACE INTO daily_'
                  'price [OK]'.format(datetime.now().strftime('%Y-%m-%d'
                                                              ' %H:%M'), num+1, company, code, len(df)))


df.intertuples() 와 관련하여

In [15]:
df = pd.DataFrame({'num_legs': [4, 2], 'num_wings': [0, 2]},
                  index=['dog', 'hawk'])

In [17]:
df

,num_legs,num_wings
dog,4,0
hawk,2,2


In [19]:
for row in df.itertuples():
    roow = row

In [21]:
type(roow)

pandas.core.frame.Pandas

In [22]:
import pandas as pd
#from bs4 import BeautifulSoup
#import urllib
#from urllib.request import urlopen
import pymysql
#import time
#import pandas.io.sql as sql
from datetime import datetime
#from threading import Timer
#import matplotlib.pyplot as plt


class MarketDB:
    def __init__(self):
        """생성자: MariaDB 연결 및 종목코드 딕셔너리 생성"""
        self.conn = pymysql.connect(host='localhost', user='root',
                                    password='Rktnek0806^^', db='INVESTAR', charset='utf8')
        self.codes = dict()
        self.getCompanyInfo()

    def __del__(self):
        """소멸자: MariaDB 연결 해제"""
        self.conn.close()

    def getCompanyInfo(self):
        """company_info 테이블에서 읽어와서 companyData와 codes에 저장"""
        sql = "SELECT * FROM company_info"
        companyInfo = pd.read_sql(sql, self.conn)
        for idx in range(len(companyInfo)):
            self.codes[companyInfo['code'].values[idx]] = companyInfo['company'].values[idx]

    def getDailyPrice(self, code, startDate, endDate):
        """daily_price 테이블에서 읽어와서 데이터프레임으로 반환"""
        sql = "SELECT * FROM daily_price WHERE code = '{}' and date >= '{}' and date <= '{}'".format(
            code, startDate, endDate)
        df = pd.read_sql(sql, self.conn)
        df.index = df['date']
        return df


In [23]:
mk = MarketDB()

In [24]:
mk.codes['018260']

'삼성에스디에스'

In [27]:
mk.getDailyPrice('삼성에스디에스', '2022-02-10', '2022-02-14')

,code,date,open,high,low,close,diff,volume
date,,,,,,,,
